In [1]:
!pip install --quiet transformers==4.1.1
!pip install --quiet pytorch-lightning==1.1.3
!pip install --quiet tokenizers==0.9.4
# !pip install Lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 33.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.7/680.7 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 17.0 MB/s eta 0:00:00


In [2]:
!pip install --upgrade pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 KB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 45.3 MB/s eta 0:00:00
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.1.3
    Uninstalling pytorch-lightning-1.1.3:
      Successfully uninstalled pytorch-lightning-1.1.3


In [3]:
import json
import numpy as np
import pandas as pd
import torch 
from pathlib import Path
from torch.utils.data import DataLoader, Dataset
# from lightning.pytorch import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap
from transformers import ( AdamW , T5ForConditionalGeneration, 
                           T5TokenizerFast as T5Tokenizer) 

In [4]:
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Global seed set to 42


42

In [5]:
!gdown --id 1AdipZtv-Dn10oAq0_sBIjf_pWr665lV-


/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1AdipZtv-Dn10oAq0_sBIjf_pWr665lV-
To: /content/QA.zip
100% 5.48M/5.48M [00:00<00:00, 250MB/s]


In [6]:
!unzip -q QA.zip

In [7]:
with Path('BioASQ/BioASQ-train-factoid-4b.json').open() as json_file:
  data = json.load(json_file)

In [8]:
questions = data['data'][0]['paragraphs']

In [9]:
data['data'][0]['title']

'BioASQ6b'

In [10]:
questions[0]

{'qas': [{'id': '52bf208003868f1b06000019_002',
   'question': 'What is the inheritance pattern of Li–Fraumeni syndrome?',
   'answers': [{'text': 'autosomal dominant', 'answer_start': 213}]}],
 'context': 'Balanced t(11;15)(q23;q15) in a TP53+/+ breast cancer patient from a Li-Fraumeni syndrome family. Li-Fraumeni Syndrome (LFS) is characterized by early-onset carcinogenesis involving multiple tumor types and shows autosomal dominant inheritance. Approximately 70% of LFS cases are due to germline mutations in the TP53 gene on chromosome 17p13.1. Mutations have also been found in the CHEK2 gene on chromosome 22q11, and others have been mapped to chromosome 11q23. While characterizing an LFS family with a documented defect in TP53, we found one family member who developed bilateral breast cancer at age 37 yet was homozygous for wild-type TP53. Her mother also developed early-onset primary bilateral breast cancer, and a sister had unilateral breast cancer and a soft tissue sarcoma. Cytog

In [11]:
def extract_question_answers(factoid_path : Path):
  with factoid_path.open() as json_file:
    data = json.load(json_file)
    questions = data['data'][0]['paragraphs']

    data_rows=[]

    for question in questions:
      context = question['context']
      for question_and_answers in question['qas']:
        question = question_and_answers['question']
        answers = question_and_answers['answers']

        for answer in answers:
          answer_text = answer['text']
          answer_start = answer['answer_start']
          answer_end = answer_start + len(answer_text)

          data_rows.append({
              'question': question,
              'context': context,
              'answer_text': answer_text,
              'answer_start': answer_start,
              "answer_end" : answer_end
          })

  return pd.DataFrame(data_rows)          

In [12]:
extract_question_answers(Path('BioASQ/BioASQ-train-factoid-4b.json')).head()

,question,context,answer_text,answer_start,answer_end
0,What is the inheritance pattern of Li–Fraumeni...,Balanced t(11;15)(q23;q15) in a TP53+/+ breast...,autosomal dominant,213,231
1,What is the inheritance pattern of Li–Fraumeni...,Genetic modeling of Li-Fraumeni syndrome in ze...,autosomal dominant,105,123
2,Which type of lung cancer is afatinib used for?,Clinical perspective of afatinib in non-small ...,EGFR-mutant NSCLC,1203,1220
3,Which hormone abnormalities are characteristic...,"DOCA sensitive pendrin expression in kidney, h...",thyroid,419,426
4,Which hormone abnormalities are characteristic...,Clinical and molecular characteristics of Pend...,thyroid,705,712


In [13]:
factoid_paths = sorted(list(Path("BioASQ/").glob("BioASQ-train-*")))
factoid_paths

[PosixPath('BioASQ/BioASQ-train-factoid-4b.json'),
 PosixPath('BioASQ/BioASQ-train-factoid-5b.json'),
 PosixPath('BioASQ/BioASQ-train-factoid-6b.json')]

In [14]:
dfs = []

for factoid_path in factoid_paths:
  dfs.append(extract_question_answers(factoid_path)) 
df =pd.concat(dfs)

In [15]:
df.head()

,question,context,answer_text,answer_start,answer_end
0,What is the inheritance pattern of Li–Fraumeni...,Balanced t(11;15)(q23;q15) in a TP53+/+ breast...,autosomal dominant,213,231
1,What is the inheritance pattern of Li–Fraumeni...,Genetic modeling of Li-Fraumeni syndrome in ze...,autosomal dominant,105,123
2,Which type of lung cancer is afatinib used for?,Clinical perspective of afatinib in non-small ...,EGFR-mutant NSCLC,1203,1220
3,Which hormone abnormalities are characteristic...,"DOCA sensitive pendrin expression in kidney, h...",thyroid,419,426
4,Which hormone abnormalities are characteristic...,Clinical and molecular characteristics of Pend...,thyroid,705,712


In [16]:
df.shape

(12988, 5)

In [17]:
df = df.drop_duplicates(subset=['context']).reset_index(drop=True)

In [18]:
df.shape

(2582, 5)

In [19]:
sample_question = df.iloc[250]
sample_question

question               Which receptor is targeted by telcagepant?
context         Randomized, controlled study of telcagepant in...
answer_text                       Calcitonin gene-related peptide
answer_start                                                  217
answer_end                                                    248
Name: 250, dtype: object

In [20]:
def color_answer(question):
  answer_start, answer_end = question['answer_start'] , question['answer_end']
  context = question['context']

  return colored(context[:answer_start], 'while') + colored(context[answer_start:answer_end + 1] , 'green') + colored(context[answer_end + 1:], 'white')


In [21]:
print(sample_question['question'])
print()
print('Answer:')

for wrap in textwrap.wrap(color_answer(sample_question),width=100):
  print(wrap)

Which receptor is targeted by telcagepant?

Answer:
Randomized, controlled study of telcagepant in patients with migraine and coronary artery disease.
OBJECTIVE: To evaluate the efficacy of telcagepant in patients with migraine and coronary artery
disease. BACKGROUND: Calcitonin gene-related peptide receptor antagonists, such as telcagepant, may
be useful for acute migraine treatment in patients with cardiovascular disease, a population for
whom triptans are contraindicated. METHODS: Randomized, double-blind, two-period (6 weeks per
period) crossover study in patients with stable coronary artery disease and migraine. Patients were
randomized 1:1 to either: (1) Period 1: telcagepant (280-mg tablet/300-mg capsule), Period 2:
acetaminophen (1000-mg); or (2) Period 1: placebo for attack 1 then acetaminophen for subsequent
attacks, Period 2: telcagepant. Patients could treat up to 12 migraine attacks per period to assess
the tolerability of telcagepant. The primary efficacy analysis evaluat

In [22]:
Model_Name = "t5-base"

In [23]:
tokenizer = T5Tokenizer.from_pretrained(Model_Name)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [24]:
sample_encoding = tokenizer( "would I rather to be feared or loved?",
                            'Easy, Both. I want people to be afraid of how much they love me')

In [25]:
sample_encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [26]:
print(sample_encoding['input_ids'])

[133, 27, 1066, 12, 36, 3, 27625, 42, 1858, 58, 1, 6844, 6, 2867, 5, 27, 241, 151, 12, 36, 7403, 13, 149, 231, 79, 333, 140, 1]


In [27]:
print(sample_encoding['attention_mask'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [28]:
preds = [
    tokenizer.decode(input_id, skip_special_tokens= True, clean_up_tokenization_spaces=True)
    for input_id in sample_encoding['input_ids']
]

In [29]:
" ".join(preds)

'would I rather to be  feared or loved ?  Easy , Both . I want people to be afraid of how much they love me '

In [30]:
encoding = tokenizer(
    sample_question['question'],
    sample_question['context'],
    max_length = 396,
    padding = 'max_length',
    truncation='only_second',
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors='pt'
)

In [31]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [32]:
tokenizer.special_tokens_map

{'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '<pad>',
 'additional_special_tokens': "['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51>', '<extra_id_52>', '<extra_i

In [33]:
tokenizer.eos_token, tokenizer.eos_token_id

('</s>', 1)

In [34]:
tokenizer.decode(encoding['input_ids'].squeeze())

'Which receptor is targeted by telcagepant?</s> Randomized, controlled study of telcagepant in patients with migraine and coronary artery disease. OBJECTIVE: To evaluate the efficacy of telcagepant in patients with migraine and coronary artery disease. BACKGROUND: Calcitonin gene-related peptide receptor antagonists, such as telcagepant, may be useful for acute migraine treatment in patients with cardiovascular disease, a population for whom triptans are contraindicated. METHODS: Randomized, double-blind, two-period (6 weeks per period) crossover study in patients with stable coronary artery disease and migraine. Patients were randomized 1:1 to either: (1) Period 1: telcagepant (280-mg tablet/300-mg capsule), Period 2: acetaminophen (1000-mg); or (2) Period 1: placebo for attack 1 then acetaminophen for subsequent attacks, Period 2: telcagepant. Patients could treat up to 12 migraine attacks per period to assess the tolerability of telcagepant. The primary efficacy analysis evaluated t

In [35]:
answer_encoding = tokenizer(
    sample_question['answer_text'],
    max_length =32,
    padding= 'max_length',
    truncation=True,
    return_attention_mask=True,
    add_special_tokens= True,
    return_tensors='pt')

In [36]:
tokenizer.decode(answer_encoding['input_ids'].squeeze())

'Calcitonin gene-related peptide</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [37]:
labels = answer_encoding['input_ids']
labels

tensor([[ 3104,    75,   155,   106,    77,  6510,    18,  3897,     3, 21826,
            15,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])

In [38]:
labels[labels == 0] = -100

In [39]:
labels

tensor([[ 3104,    75,   155,   106,    77,  6510,    18,  3897,     3, 21826,
            15,     1,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100]])

In [40]:
from os import truncate
class BioDataset(Dataset):

  def __init__(
      self,
      data: pd.DataFrame,
      tokenizer: T5Tokenizer,
      source_max_token_len: int=396,
      target_max_token_len: int = 32
  ):
    self.tokenizer = tokenizer
    self.data = data
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row=self.data.iloc[index]
    source_encoding = tokenizer(
      data_row['question'],
      data_row['context'],
      max_length = self.source_max_token_len,
      padding = 'max_length',
      truncation = 'only_second',
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors='pt'        
    )

  
    target_encoding = tokenizer(
      data_row['answer_text'],
      max_length =self.target_max_token_len,
      padding= 'max_length',
      truncation=True,
      return_attention_mask=True,
      add_special_tokens= True,
      return_tensors='pt'
    )

    labels = target_encoding['input_ids']
    labels[labels == 0] = -100
    return dict(
        question=data_row['question'],
        context=data_row['context'],
        answer_text = data_row['answer_text'],
        input_ids = source_encoding['input_ids'].flatten(),
        attention_mask=source_encoding['attention_mask'].flatten(),
        labels=labels.flatten()
    )
    

In [41]:
sample_dataset = BioDataset(df,tokenizer)

In [42]:
for data in sample_dataset:
  print(data['question'])
  print(data['answer_text'])
  print(data['input_ids'][:10])
  print(data['labels'][:10])
  break

What is the inheritance pattern of Li–Fraumeni syndrome?
autosomal dominant
tensor([  363,    19,     8, 28915,  3275,    13,  1414,   104,   371,  6340])
tensor([ 1510, 10348,   138, 12613,     1,  -100,  -100,  -100,  -100,  -100])


In [43]:
train_df, val_df = train_test_split(df,test_size=0.05)

In [44]:
train_df.shape, val_df.shape

((2452, 5), (130, 5))

In [45]:
class BioQADataModel(pl.LightningDataModule):

  def __init__(
      self,
      train_df: pd.DataFrame,
      test_df: pd.DataFrame,
      tokenizer: T5Tokenizer,
      batch_size: int=8,
      source_max_token_len: int = 396,
      target_max_token_len: int =32
  ):
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len


  def setup(self,stage=None):
    self.train_dataset = BioDataset(
        self.train_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
    )

    self.test_dataset = BioDataset(
        self.test_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
    )

  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size = self.batch_size,
        shuffle=True,
        num_workers=4
    )
  
  def val_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size = 1,
        num_workers=4
    )

  def test_dataloader(self):
    return DataLoader(
        self.test_daset,
        batch_size = 1,
        num_workers=4
    )
  

In [46]:
BATCH_SIZE =8
N_EPOCHS = 6

data_module = BioQADataModel(train_df, val_df,tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

In [47]:
# model = T5ForConditionalGeneration.from_pretrained(Model_Name, return_dict=True)

In [48]:
# input_ids = tokenizer(
#     "Translate English to German: I talk a lot, so I've learned to tune myself out ",
#     return_tensors='pt'
# ).input_ids

# generated_ids = model.generate(input_ids = input_ids)
# generated_ids

In [49]:
# pred = [
#     tokenizer.decode(gen_id, skip_special_tokens = True, clean_up_tokenization_space = True)
#                      for gen_id in generated_ids
# ]

In [50]:
# pred

In [51]:
# " ".join(pred)

In [52]:
# output = model(
#     input_ids = encoding["input_ids"],
#     attention_mask=encoding["attention_mask"],
#     labels = labels
# )

In [53]:
# output.logits.shape

In [54]:
# output.loss

Modeling

In [55]:

from torch.nn.modules import loss
class BioModel(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(Model_Name, return_dict=True)
  
  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels 
    )

    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    loss,outputs = self(input_ids, attention_mask, labels)
    self.log('train_loss', loss, prog_bar=True, logger=True)
    return loss


  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch["attention_mask"]
    labels = batch['labels']
    loss,outputs = self(input_ids, attention_mask ,labels)
    self.log('val_loss', loss, prog_bar=True, logger=True)
    return loss


  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    loss,outputs  = self(input_ids, attention_mask, labels)
    self.log('test_loss', loss, prog_bar=True, logger=True)
    return loss

  def configure_optimizers(self):
    return AdamW(self.parameters(), lr=0.0001)


In [56]:
model = BioModel()

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [57]:
model

BioModel(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseReluDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_features=

In [58]:
# from transformers.models.t5.modeling_t5 import checkpoint

checkpoint_callback = ModelCheckpoint(
    dirpath = 'checkpoints',
    filename= 'best_chackpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min'
)

In [59]:
trainer = pl.Trainer(
    callbacks = [checkpoint_callback],
    max_epochs=N_EPOCHS,
    # enable_progress_bar=True,
    accelerator="gpu")
    # progress_bar_refresh_rate=30)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [60]:
%load_ext tensorboard

In [61]:
# !rm -rf lightning_logs
trainer.fit(model,data_module)

INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 307: 'val_loss' reached 0.28404 (best 0.28404), saving model to '/content/checkpoints/best_chackpoint.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 1228: 'val_loss' reached 0.22377 (best 0.22377), saving model to '/content/checkpoints/best_chackpoint.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 1535: 'val_loss' reached 0.21271 (best 0.21271), saving model to '/content/checkpoints/best_chackpoint.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 5, global step 1842: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=6` reached.


In [62]:
trained_model = BioModel.load_from_checkpoint('/content/checkpoints/best_chackpoint.ckpt')

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [63]:
trained_model.freeze()

In [64]:

def generate_answer(question):
  source_encoding =  tokenizer(
      question['question'],
      question['context'],
      max_length = 396,
      padding = 'max_length',
      truncation= 'only_second',
      return_attention_mask=True,
      add_special_tokens = True,
      return_tensors = 'pt'
  )

  generate_ids = trained_model.model.generate(
    input_ids=source_encoding['input_ids'],
    attention_mask = source_encoding['attention_mask'],
    num_beams= 1,
    max_length=80,
    repetition_penalty=2.5,
    length_penalty=1.0,
    early_stopping=True,
    use_cache = True
)
  
  pre = [
      tokenizer.decode(generated_id ,skip_special_tokens=True, clean_up_tokenization_spaces=True)
      for generated_id in generate_ids 
  ]

  return " ".join(pre)

In [65]:
sample_question = val_df.iloc[5]

In [66]:
sample_question['question']

'Which enzyme is inhibited by ixazomib?'

In [67]:
sample_question['answer_text']

'proteasome'

In [68]:
generate_answer(sample_question)

'proteasome'

In [69]:
sample_question = val_df.iloc[10]

In [70]:
sample_question['question']

'Which gene mutations cause the Marfan syndrome?'

In [71]:
sample_question['answer_text']

'FBN1'

In [73]:
generate_answer(sample_question)

'FBN1'